<a href="https://colab.research.google.com/github/Eliemulolo/E-leaning-web-app/blob/main/elie_mulolo_70600_colab1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install Java (JVM), required by Spark
!apt-get install openjdk-8-jdk-headless -qq





E: Failed to fetch http://security.ubuntu.com/ubuntu/pool/universe/o/openjdk-8/openjdk-8-jre-headless_8u462-ga%7eus1-0ubuntu2%7e22.04.2_amd64.deb  404  Not Found [IP: 91.189.91.83 80]
E: Failed to fetch http://security.ubuntu.com/ubuntu/pool/universe/o/openjdk-8/openjdk-8-jdk-headless_8u462-ga%7eus1-0ubuntu2%7e22.04.2_amd64.deb  404  Not Found [IP: 91.189.91.83 80]
E: Unable to fetch some archives, maybe run apt-get update or try with --fix-missing?


In [ ]:
# Download Spark (comes with Hadoop inside)
!wget https://dlcdn.apache.org/spark/spark-3.2.1/spark-3.2.1-bin-hadoop3.2.tgz

# Unpack the archive
!tar -xvf spark-3.2.1-bin-hadoop3.2.tgz

# Move Spark to /content/spark for easier access
!mv spark-3.2.1-bin-hadoop3.2 /content/spark


--2025-12-02 16:39:11--  https://dlcdn.apache.org/spark/spark-3.2.1/spark-3.2.1-bin-hadoop3.2.tgz
Resolving dlcdn.apache.org (dlcdn.apache.org)... 151.101.2.132, 2a04:4e42::644
Connecting to dlcdn.apache.org (dlcdn.apache.org)|151.101.2.132|:443... connected.
HTTP request sent, awaiting response... 404 Not Found
2025-12-02 16:39:11 ERROR 404: Not Found.

tar: spark-3.2.1-bin-hadoop3.2.tgz: Cannot open: No such file or directory
tar: Error is not recoverable: exiting now
mv: cannot stat 'spark-3.2.1-bin-hadoop3.2': No such file or directory


In [ ]:
# Install required Python libraries for Spark
!pip install findspark pyspark py4j


In [ ]:
# Import OS and SYS (managing system paths)
import os, sys


In [ ]:
# Import findspark, used to detect Spark installed manually
import findspark


In [ ]:
# Initialize findspark → makes Spark visible to Python
findspark.init()

# Show the path where Spark was found
findspark.find()


'/usr/local/lib/python3.12/dist-packages/pyspark'

In [ ]:
# Import main PySpark modules
import pyspark
from pyspark.sql import DataFrame, SparkSession  # DataFrame tools + Spark Session
from typing import List                          # Used for type annotations


In [ ]:
# Import Spark SQL types (StringType, IntegerType, etc.)
import pyspark.sql.types as T


In [ ]:
# Import Spark SQL functions (sum, count, desc, col, etc.)
import pyspark.sql.functions as F


In [ ]:
# Create a Spark session (the main entry point to Spark)
sparkXY = SparkSession.builder.appName("First Surname Spark Example").getOrCreate()


In [ ]:
# Display Spark information (version, configuration, etc.)
sparkXY


In [ ]:
# Define URL of COVID dataset (CSV)
path = "https://raw.githubusercontent.com/owid/covid-19-data/master/public/data/owid-covid-data.csv"


In [ ]:
# Download the file using HTTP GET request
import requests
req = requests.get(path)


In [ ]:
# Extract raw content (bytes) from the downloaded request
url_content = req.content


In [ ]:
# Choose a name for the local CSV file
csv_file_name = "owid-covid-data.csv"


In [ ]:
# Create the file locally and save the downloaded bytes
csv_file = open(csv_file_name, "wb")
csv_file.write(url_content)
csv_file.close()


In [ ]:
# Read the CSV into a Spark DataFrame
# header=True → first row contains column names
# inferSchema=True → Spark detects column types automatically
df_XY = sparkXY.read.csv("/content/" + csv_file_name, header=True, inferSchema=True)


In [ ]:
# Print the structure of the DataFrame (columns + types)
df_XY.printSchema()


root
 |-- iso_code: string (nullable = true)
 |-- continent: string (nullable = true)
 |-- location: string (nullable = true)
 |-- date: date (nullable = true)
 |-- total_cases: integer (nullable = true)
 |-- new_cases: integer (nullable = true)
 |-- new_cases_smoothed: double (nullable = true)
 |-- total_deaths: integer (nullable = true)
 |-- new_deaths: integer (nullable = true)
 |-- new_deaths_smoothed: double (nullable = true)
 |-- total_cases_per_million: double (nullable = true)
 |-- new_cases_per_million: double (nullable = true)
 |-- new_cases_smoothed_per_million: double (nullable = true)
 |-- total_deaths_per_million: double (nullable = true)
 |-- new_deaths_per_million: double (nullable = true)
 |-- new_deaths_smoothed_per_million: double (nullable = true)
 |-- reproduction_rate: double (nullable = true)
 |-- icu_patients: integer (nullable = true)
 |-- icu_patients_per_million: double (nullable = true)
 |-- hosp_patients: integer (nullable = true)
 |-- hosp_patients_per_mil

In [ ]:
# Display statistical summary of numerical columns
df_XY.describe().show()


+-------+--------+-------------+-----------+--------------------+------------------+------------------+------------------+------------------+-------------------+-----------------------+---------------------+------------------------------+------------------------+----------------------+-------------------------------+------------------+------------------+------------------------+------------------+-------------------------+---------------------+---------------------------------+----------------------+----------------------------------+-------------------+------------------+------------------------+----------------------+------------------+-------------------------------+-------------------+------------------+-------------+--------------------+--------------------+-----------------------+--------------------+------------------+-------------------------+------------------------------+-----------------------------+-----------------------------------+--------------------------+-------------

In [ ]:
# Filter rows where location == "United States"
# Sort results by latest date (descending order)
df_XY.filter(df_XY.location == "United States").orderBy(F.desc("date")).show()


+--------+-------------+-------------+----------+-----------+---------+------------------+------------+----------+-------------------+-----------------------+---------------------+------------------------------+------------------------+----------------------+-------------------------------+-----------------+------------+------------------------+-------------+-------------------------+---------------------+---------------------------------+----------------------+----------------------------------+-----------+---------+------------------------+----------------------+------------------+-------------------------------+-------------+--------------+-----------+------------------+-----------------+-----------------------+--------------+----------------+-------------------------+------------------------------+-----------------------------+-----------------------------------+--------------------------+-------------------------------------+------------------------------+-------------------------

In [ ]:
# Group by country → sum new_cases
# Sort countries by highest total number of new cases
df_XY.groupBy("location").sum("new_cases").orderBy(F.desc("sum(new_cases)")).show(truncate=False)


+-----------------------------+--------------+
|location                     |sum(new_cases)|
+-----------------------------+--------------+
|World                        |775935057     |
|High-income countries        |429044052     |
|Asia                         |301564180     |
|Europe                       |252916868     |
|Upper-middle-income countries|251756125     |
|European Union (27)          |185822587     |
|North America                |124492698     |
|United States                |103436829     |
|China                        |99373219      |
|Lower-middle-income countries|92019711      |
|South America                |68811012      |
|India                        |45041748      |
|France                       |38997490      |
|Germany                      |38437756      |
|Brazil                       |37511921      |
|South Korea                  |34571873      |
|Japan                        |33803572      |
|Italy                        |26781078      |
|United Kingd

In [ ]:
# Register DataFrame as a SQL temporary table
df_XY.createOrReplaceTempView("covid_data")


In [ ]:
# Read entire table using SQL
df2 = sparkXY.sql("SELECT * FROM covid_data")


In [ ]:
# Display schema of df2 (same as original)
df2.printSchema()


root
 |-- iso_code: string (nullable = true)
 |-- continent: string (nullable = true)
 |-- location: string (nullable = true)
 |-- date: date (nullable = true)
 |-- total_cases: integer (nullable = true)
 |-- new_cases: integer (nullable = true)
 |-- new_cases_smoothed: double (nullable = true)
 |-- total_deaths: integer (nullable = true)
 |-- new_deaths: integer (nullable = true)
 |-- new_deaths_smoothed: double (nullable = true)
 |-- total_cases_per_million: double (nullable = true)
 |-- new_cases_per_million: double (nullable = true)
 |-- new_cases_smoothed_per_million: double (nullable = true)
 |-- total_deaths_per_million: double (nullable = true)
 |-- new_deaths_per_million: double (nullable = true)
 |-- new_deaths_smoothed_per_million: double (nullable = true)
 |-- reproduction_rate: double (nullable = true)
 |-- icu_patients: integer (nullable = true)
 |-- icu_patients_per_million: double (nullable = true)
 |-- hosp_patients: integer (nullable = true)
 |-- hosp_patients_per_mil

In [ ]:
# Show first rows of df2
df2.show()


+--------+---------+-----------+----------+-----------+---------+------------------+------------+----------+-------------------+-----------------------+---------------------+------------------------------+------------------------+----------------------+-------------------------------+-----------------+------------+------------------------+-------------+-------------------------+---------------------+---------------------------------+----------------------+----------------------------------+-----------+---------+------------------------+----------------------+------------------+-------------------------------+-------------+--------------+-----------+------------------+-----------------+-----------------------+--------------+----------------+-------------------------+------------------------------+-----------------------------+-----------------------------------+--------------------------+-------------------------------------+------------------------------+-------------------------------

In [ ]:
# SQL group-by: count number of rows per country
groupDF = sparkXY.sql("SELECT location, COUNT(*) FROM covid_data GROUP BY location")
groupDF.show()


+--------------------+--------+
|            location|count(1)|
+--------------------+--------+
|                Chad|    1674|
|            Anguilla|    1674|
|            Kiribati|    1674|
|              Guyana|    1674|
|             Eritrea|    1674|
|              Jersey|    1674|
|            Djibouti|    1674|
|                Fiji|    1674|
|                Iraq|    1674|
|              Europe|    1684|
|             Germany|    1674|
|             Comoros|    1674|
|         Afghanistan|    1674|
|            Cambodia|    1674|
|High-income count...|    3026|
|              Jordan|    1674|
|              France|    1674|
|              Greece|    1674|
|              Kosovo|    1674|
|              Africa|    1674|
+--------------------+--------+
only showing top 20 rows

